In [1]:
!pip install pydicom

In [2]:
from apache_beam.io.gcp import gcsio
from google.cloud import storage
from pathlib import Path

import tensorflow as tf
import numpy as np

import pydicom as dcm
import PIL
from PIL import Image as pilim
import matplotlib

import skimage.color
import skimage.filters
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import os
import apache_beam as beam
import subprocess
import logging
import sys

from os import listdir
import matplotlib.pyplot as plt

In [3]:
def resolve_gcs_uri_to_local_uri(gcs_file_uri: str) -> str:
    """
    Use an existing gcsfuse mountpoint, or use gcsfuse to mount a bucket
    locally if not mountpoint exists, to get a local filepath for a file stored
    in a GCS bucket.

    Args:
        gcs_file_uri (str): A "gs:/..." URI for a file stored in a GCS bucket.

    Returns:
        str: A string indicating a local filepath where gcs_file_uri can be
        found.
    """
    if not gcs_file_uri.startswith("gs:/"):
        raise Exception('Please use a "gs:/" GCS path.')

    # Pathlib isn't wholly apposite to use for URLs (specifically, it changes
    # 'gs://...' to 'gs:/...'), but it's useful in this case:
    gcs_path = Path(gcs_file_uri)
    bucket_name = gcs_path.parts[1]
    file_remaining_path = Path(*gcs_path.parts[2:])
    logging.info("Checking for existing mountpoint for %s...", bucket_name)
    existing_mountpoint_check = subprocess.run(
        f'findmnt --noheadings --output TARGET --source "{bucket_name}"',
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
    )
    existing_mountpoint_check_stdout = existing_mountpoint_check.stdout.decode().strip()

    if existing_mountpoint_check.returncode == 127:
        raise Exception(
            f'Error when running findmnt: "{existing_mountpoint_check_stdout}"'
        )

    if (
        existing_mountpoint_check.returncode == 0
        and existing_mountpoint_check_stdout != ""
    ):
        # An existing mountpoint for the bucket exists; we will reuse it here,
        # if it contains the file we're looking for (it's possible that the
        # bucket is mounted, but to a different subdirectory within the bucket
        # than we need).
        for mountpoint in existing_mountpoint_check_stdout.split("\n"):
            logging.info(
                'Checking for file "%s" at "%s"...', file_remaining_path, mountpoint
            )
            if Path(mountpoint, file_remaining_path).exists():
                found_file_path = os.path.join(mountpoint, file_remaining_path)
                logging.info('Found file at "%s"', found_file_path)
                return found_file_path

    # No existing mountpoint exists, so we will mount the bucket with gcsfuse:
    import tempfile

    tmp_directory = tempfile.mkdtemp()
    logging.info('Creating new mountpoint at "%s"...', tmp_directory)
    mount_command = subprocess.run(
        # Unexpectedly, using `f"gcsfuse --implicit-dirs --only-dir {specific_bucket_directory} {bucket_name} {tmp_directory}"`
        # resulted in a mountpoint that was *unusably slow* when trying to,
        # e.g., using `.get_thumbnail()`. Thus, here, we are not using
        # `--only-dir.`
        f"gcsfuse --implicit-dirs {bucket_name} {tmp_directory}",
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
    )
    mount_command_stdout = mount_command.stdout.decode().strip()
    if "File system has been successfully mounted" not in mount_command_stdout:
        raise Exception(
            f'Error when running gcsfuse: "{mount_command_stdout}". Exit code was {existing_mountpoint_check.returncode}.'
        )

    return os.path.join(tmp_directory, file_remaining_path)

In [4]:
gcs_data_path = "gs://rsna_hemorrhage/dataset/stage_2_train"

In [5]:
gcsfuse_mount_data_path = resolve_gcs_uri_to_local_uri(gcs_file_uri=gcs_data_path)

In [6]:
gcsfuse_mount_data_path

'/var/tmp/tmp0vbsk5z1/dataset/stage_2_train'

In [7]:
os.listdir(gcsfuse_mount_data_path)[0]

'ID_000012eaf.dcm'

In [8]:
# Transform the pixel values to the Hounsfield units, we can achieve this using the headers in the medical_image file.
# We will use the Rescale Intercept and Rescale Slope headers.
def transform_to_hu(medical_image, image):
    intercept = medical_image.RescaleIntercept
    slope = medical_image.RescaleSlope
    hu_image = image * slope + intercept
    return hu_image
# If we want a specific zone of the image we can window the image:
def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    return window_image




In [9]:
#import pandas as pd
#df=pd.DataFrame(DCMFiles_train)

In [10]:
#df.to_csv('train_dcm_names.csv') 

In [11]:
names=['ID_cc05c6036.dcm',
'ID_cad934fd1.dcm',
'ID_9015ebad3.dcm',
'ID_d9bb13001.dcm',
'ID_097887f49.dcm',
'ID_0f327294a.dcm',
'ID_be946a368.dcm',
'ID_c7ef74ad5.dcm',
'ID_b5f86ee31.dcm',
'ID_273fc32b3.dcm',
'ID_f8d83d01d.dcm',
'ID_5e7b353e0.dcm',
'ID_e7fe2dfc8.dcm',
'ID_6e38d2116.dcm',
'ID_80424a7fe.dcm',
'ID_17675629b.dcm',
'ID_b266f2572.dcm',
'ID_6407ea995.dcm',
'ID_183790b20.dcm',
'ID_c8261bb37.dcm',
'ID_e9288cec6.dcm',
'ID_d55184488.dcm',
'ID_2eafafdb5.dcm',
'ID_e62c94757.dcm',
'ID_c5da92217.dcm',
'ID_1bb6d2348.dcm',
'ID_789af0490.dcm',
'ID_f6c08c525.dcm',
'ID_9d00d4110.dcm',
'ID_fb3426d72.dcm',
'ID_c06198ecc.dcm',
'ID_d27769b51.dcm',
'ID_0905e903d.dcm',
'ID_6db66539f.dcm',
'ID_78e042ccd.dcm',
'ID_294e07a13.dcm',
'ID_26bb85b5d.dcm',
'ID_d48cabb50.dcm',
'ID_3d7bb4b3f.dcm',
'ID_5b23f7ca0.dcm',
'ID_de77d6bcf.dcm',
'ID_1def6749b.dcm',
'ID_05557fc2b.dcm',
'ID_d02c23d3e.dcm',
'ID_126560c3f.dcm',
'ID_5e05922be.dcm',
'ID_803bb9917.dcm',
'ID_846002587.dcm',
'ID_ab822e75e.dcm',
'ID_7c8a46b38.dcm',
'ID_5eeed619e.dcm',
'ID_1a793dfaa.dcm',
'ID_fce27e580.dcm',
'ID_4f3bd0fb6.dcm',
'ID_45574291e.dcm',
'ID_51ef8d46c.dcm',
'ID_30b4e6c96.dcm',
'ID_8c933637f.dcm',
'ID_3bc8e7e09.dcm',
'ID_1cdf2ddfa.dcm',
'ID_189e27b9d.dcm',
'ID_91cf148ae.dcm',
'ID_4793f3cbd.dcm',
'ID_a74b33fab.dcm',
'ID_8c43bd0f1.dcm',
'ID_345b03e2f.dcm',
'ID_b4e205b79.dcm',
'ID_38c735b13.dcm',
'ID_cf047e208.dcm',
'ID_90b2c669d.dcm',
'ID_b6a67c908.dcm',
'ID_6d0207c4b.dcm',
'ID_4768283fe.dcm',
'ID_aa3c8b03b.dcm',
'ID_066faa1e3.dcm',
'ID_22d84264e.dcm',
'ID_c1e359fa9.dcm',
'ID_c2010d32c.dcm',
'ID_a48913b44.dcm',
'ID_94e67cbad.dcm',
'ID_370ee3d29.dcm',
'ID_eb4adc536.dcm',
'ID_754dc144d.dcm',
'ID_85014fd52.dcm',
'ID_b3e0e7265.dcm',
'ID_a67ae96d8.dcm',
'ID_b40539cdf.dcm',
'ID_de83c51b9.dcm',
'ID_256cc4d17.dcm',
'ID_3cda88a55.dcm',
'ID_56d978968.dcm',
'ID_14126f313.dcm',
'ID_5f68a12b7.dcm',
'ID_19ac55310.dcm',
'ID_87db3ca23.dcm',
'ID_1e19fd061.dcm',
'ID_2f4287f67.dcm',
'ID_1f84c2fc3.dcm',
'ID_f44d31152.dcm',
'ID_ebb84d23c.dcm',
'ID_05f93b94c.dcm',
'ID_c2033c9a3.dcm',
'ID_296e5eb0c.dcm',
'ID_2991acdcb.dcm',
'ID_0ecfaf34f.dcm',
'ID_0dab76891.dcm',
'ID_1f8b66c62.dcm',
'ID_8198082e7.dcm',
'ID_168936a51.dcm',
'ID_8264d1b16.dcm',
'ID_affe9e7c5.dcm',
'ID_2ccd38a91.dcm',
'ID_b53498295.dcm',
'ID_48f98eaf6.dcm',
'ID_f058aa7de.dcm',
'ID_bc428a5b3.dcm',
'ID_cc5e8b1a1.dcm',
'ID_fcd18525c.dcm',
'ID_8270c2e11.dcm',
'ID_675a34a3f.dcm',
'ID_35b385865.dcm',
'ID_f442fa820.dcm',
'ID_236ea9f17.dcm',
'ID_44761f893.dcm',
'ID_4ead8e3c3.dcm',
'ID_67695b674.dcm',
'ID_d8c4e6b71.dcm',
'ID_a8747b038.dcm',
'ID_80cb6458a.dcm',
'ID_90072ab7d.dcm',
'ID_7f275fc83.dcm',
'ID_07d3e686f.dcm',
'ID_2c36cc6db.dcm',
'ID_ce426baa7.dcm',
'ID_72100e1b2.dcm',
'ID_d68a25d81.dcm',
'ID_fd7588f3f.dcm',
'ID_1cf2e8bc4.dcm',
'ID_f6eef24d7.dcm',
'ID_0b6b62108.dcm',
'ID_79cdd056e.dcm',
'ID_90f50ff88.dcm',
'ID_8c400f093.dcm',
'ID_abe962954.dcm',
'ID_85f32a72a.dcm',
'ID_c7f1f388f.dcm',
'ID_bace48e8d.dcm',
'ID_b44ac17d6.dcm',
'ID_b19716af9.dcm',
'ID_9af09fc39.dcm',
'ID_4f6a16276.dcm',
'ID_7c48a6291.dcm',
'ID_666e04902.dcm',
'ID_558330cc1.dcm',
'ID_f52d1a434.dcm',
'ID_b0310ef5a.dcm',
'ID_b2f4305e7.dcm',
'ID_133ac4488.dcm',
'ID_ca04fff3c.dcm',
'ID_7e9fac4c0.dcm',
'ID_6e8020a95.dcm',
'ID_34faddfe5.dcm',
'ID_1718cd26b.dcm',
'ID_ef950cc67.dcm',
'ID_86b672c58.dcm',
'ID_07e3ff702.dcm',
'ID_6bb488b7d.dcm',
'ID_93691a79b.dcm',
'ID_473ed4316.dcm',
'ID_3839296de.dcm',
'ID_f62315b56.dcm',
'ID_150763f9a.dcm',
'ID_9b5f7ea2b.dcm',
'ID_bb7021961.dcm',
'ID_b04deed8a.dcm',
'ID_dd7fd876e.dcm',
'ID_9da280604.dcm',
'ID_c40866f75.dcm',
'ID_27df6ccfc.dcm',
'ID_268c308c1.dcm',
'ID_5b59c0dd2.dcm',
'ID_7fb3e87dc.dcm',
'ID_2ce249e2b.dcm',
'ID_6b46adf91.dcm',
'ID_bc90468b8.dcm',
'ID_5a59e27d3.dcm',
'ID_c2acd0b53.dcm',
'ID_14f5dbac8.dcm',
'ID_dd486f46a.dcm',
'ID_36fae7134.dcm',
'ID_279335f37.dcm',
'ID_cd640ac76.dcm',
'ID_6497e3c22.dcm',
'ID_91daeb538.dcm',
'ID_9060a4c4f.dcm',
'ID_84b26ffea.dcm',
'ID_64a40e32f.dcm',
'ID_38d5446d5.dcm',
'ID_8baf1fe7e.dcm',
'ID_d11c430c8.dcm',
'ID_c1b34dba2.dcm',
'ID_ed66086cb.dcm',
'ID_89d3b4a18.dcm',
'ID_17815bf09.dcm',
'ID_55bba295b.dcm',
'ID_56a504ef9.dcm',
'ID_856b4b6f3.dcm',
'ID_49669cf87.dcm',
'ID_cfefa676c.dcm',
'ID_487606379.dcm',
'ID_1d6274da9.dcm',
'ID_05177b99a.dcm',
'ID_010e3ab5f.dcm',
'ID_683b0aa9c.dcm',
'ID_328c5f5ee.dcm',
'ID_b4768ef41.dcm',
'ID_2914a8624.dcm',
'ID_95842ea95.dcm',
'ID_b038f9b44.dcm',
'ID_0e375c2bc.dcm',
'ID_ededea385.dcm',
'ID_dde4ec792.dcm',
'ID_4a98ad98a.dcm',
'ID_8a98a7f06.dcm',
'ID_c3b3fa8dc.dcm',
'ID_2b674b07f.dcm',
'ID_2f08babcf.dcm',
'ID_15731c542.dcm',
'ID_09cf307d5.dcm',
'ID_eea453ce2.dcm',
'ID_fce3ee648.dcm',
'ID_ffa5c9e44.dcm',
'ID_80d578fd1.dcm',
'ID_5bd28d374.dcm',
'ID_4b2590181.dcm',
'ID_9d08221cb.dcm',
'ID_c9e12cf17.dcm',
'ID_69ed2871b.dcm',
'ID_17099236f.dcm',
'ID_2ae19eac3.dcm',
'ID_cac3ba664.dcm',
'ID_a267d6d46.dcm',
'ID_a15b145e7.dcm',
'ID_dd73c869c.dcm',
'ID_e74f3ec30.dcm',
'ID_288bdd0d9.dcm',
'ID_331fd51b7.dcm',
'ID_7cce4a847.dcm',
'ID_0a514cda5.dcm',
'ID_f420aabe4.dcm',
'ID_09738340a.dcm',
'ID_95378f116.dcm',
'ID_e83080c37.dcm',
'ID_6e9d71d64.dcm',
'ID_ee478bfa3.dcm',
'ID_118162996.dcm',
'ID_2aef29716.dcm',
'ID_6861b533a.dcm',
'ID_11fb6606a.dcm',
'ID_0476c55ec.dcm',
'ID_fe0964e6f.dcm',
'ID_549b52f64.dcm',
'ID_8d6161c36.dcm',
'ID_04ede913c.dcm',
'ID_2e513d62e.dcm',
'ID_a5c3f4001.dcm',
'ID_0a54daf86.dcm',
'ID_9f0f919c0.dcm',
'ID_177baa70e.dcm',
'ID_6e6482ccb.dcm',
'ID_b43e38ab3.dcm',
'ID_703d9680a.dcm',
'ID_6cfa6f5db.dcm',
'ID_9496a8fa2.dcm',
'ID_2ee19ae00.dcm',
'ID_bc8f3946e.dcm',
'ID_8d307285e.dcm',
'ID_a3b4b4c51.dcm',
'ID_18bbc4a12.dcm',
'ID_d194a9986.dcm',
'ID_ded233be0.dcm',
'ID_c6bc364ec.dcm',
'ID_cdb7b243a.dcm',
'ID_7ad84e99b.dcm',
'ID_12825b334.dcm',
'ID_245d94051.dcm',
'ID_b2e89d13c.dcm',
'ID_1fe88ce87.dcm',
'ID_1fa5213a0.dcm',
'ID_7eac302fe.dcm',
'ID_84cc50039.dcm',
'ID_da380b786.dcm',
'ID_bd9410e16.dcm',
'ID_3a41116a2.dcm',
'ID_f84f4c05c.dcm',
'ID_669857408.dcm',
'ID_22f4efde3.dcm',
'ID_e1fc9bf9b.dcm',
'ID_e18fe4557.dcm',
'ID_68dfe9785.dcm',
'ID_bbdc6200e.dcm',
'ID_1e13e724b.dcm',
'ID_317e34531.dcm',
'ID_95c19b19d.dcm',
'ID_7ae3ae3c7.dcm',
'ID_2ea2a1211.dcm',
'ID_7c9eba091.dcm',
'ID_7ef429754.dcm',
'ID_4e808724b.dcm',
'ID_517fab4ca.dcm',
'ID_590cde462.dcm',
'ID_ffcde7b6c.dcm',
'ID_32a781b0a.dcm',
'ID_cd9ccb3e1.dcm',
'ID_7c6319eec.dcm',
'ID_18ea9c105.dcm',
'ID_d7ece87ea.dcm',
'ID_72dc8f2c2.dcm',
'ID_003cfdbf7.dcm',
'ID_508a92f33.dcm',
'ID_2ccb6e996.dcm',
'ID_67521aaeb.dcm',
'ID_99ab10812.dcm',
'ID_7da1f2641.dcm',
'ID_df189c550.dcm',
'ID_2aaf12177.dcm',
'ID_b8ee5f7c3.dcm',
'ID_b69e8283b.dcm',
'ID_041f91037.dcm',
'ID_1debd2c5a.dcm',
'ID_954141f29.dcm',
'ID_f61e059d5.dcm',
'ID_3a6ad12f9.dcm',
'ID_adcaaafba.dcm',
'ID_cd44ca400.dcm',
'ID_b56bee98c.dcm',
'ID_b0b13bf15.dcm',
'ID_368ee28f5.dcm',
'ID_617527ffa.dcm',
'ID_ebc52fc04.dcm',
'ID_99891a9ea.dcm',
'ID_2404a9e12.dcm',
'ID_a90256d5b.dcm',
'ID_b7f5c4073.dcm',
'ID_802f66b11.dcm',
'ID_28b30ea70.dcm',
'ID_df76dda3c.dcm',
'ID_7fdc35957.dcm',
'ID_fc2200ea0.dcm',
'ID_1115cf9c5.dcm',
'ID_752354810.dcm',
'ID_91b491f65.dcm',
'ID_dfa100c05.dcm',
'ID_07c8a8e00.dcm',
'ID_1a53db280.dcm',
'ID_7bb86c23a.dcm',
'ID_e7e2d936b.dcm',
'ID_cb20f9b7d.dcm',
'ID_d38226b6b.dcm',
'ID_935627e80.dcm',
'ID_990dc0fa1.dcm',
'ID_84774040f.dcm',
'ID_c4a471dc0.dcm',
'ID_2794a2c58.dcm',
'ID_4ac1d0f77.dcm',
'ID_a4abc92cd.dcm',
'ID_cdb6d7b8b.dcm',
'ID_17afc4add.dcm',
'ID_e8f8b0e4c.dcm',
'ID_a47904a31.dcm',
'ID_bc9cac813.dcm',
'ID_eee3a7a07.dcm',
'ID_143794f12.dcm',
'ID_82c0204b0.dcm',
'ID_7e229a46d.dcm',
'ID_0ec758cee.dcm',
'ID_7a2384c42.dcm',
'ID_cad03375a.dcm',
'ID_14a07ae22.dcm',
'ID_914d7e49f.dcm',
'ID_ae40abc52.dcm',
'ID_23cbfa662.dcm',
'ID_3ed461bb2.dcm',
'ID_c818a4a04.dcm',
'ID_4f0bfe3b5.dcm',
'ID_11e3e8dba.dcm',
'ID_293b8a280.dcm',
'ID_70eb335d6.dcm',
'ID_4209e06fe.dcm',
'ID_7f28fc1c0.dcm',
'ID_c97108c2f.dcm',
'ID_7517b4926.dcm',
'ID_084bbfea5.dcm',
'ID_f0dad1ede.dcm',
'ID_26faf7396.dcm',
'ID_ca2b1278c.dcm',
'ID_0eabca49f.dcm',
'ID_f6bdbf737.dcm',
'ID_6c4dabf7e.dcm',
'ID_c7e4f40b4.dcm',
'ID_2c8a377ac.dcm',
'ID_3cd73c996.dcm',
'ID_881cca4c8.dcm',
'ID_1554c6897.dcm',
'ID_6721a4aaa.dcm',
'ID_1b2818e0c.dcm',
'ID_a4bed6aaa.dcm',
'ID_774ec6f93.dcm',
'ID_23cdc1a1e.dcm',
'ID_077888e44.dcm',
'ID_bbab9c348.dcm',
'ID_9d8d8f128.dcm',
'ID_8a5478e11.dcm',
'ID_e52bee931.dcm',
'ID_8f254b00f.dcm',
'ID_e5b1cec45.dcm',
'ID_14a909134.dcm',
'ID_5cc17f001.dcm',
'ID_1e6d2dcb3.dcm',
'ID_1696a8073.dcm',
'ID_f39224145.dcm',
'ID_b551a1bc4.dcm',
'ID_6831a51d3.dcm',
'ID_f5364ac85.dcm',
'ID_3c8d28ea7.dcm',
'ID_7e80ed4a0.dcm',
'ID_394fefe1d.dcm',
'ID_46b08363d.dcm',
'ID_4990ded63.dcm',
'ID_10f022590.dcm',
'ID_0befb08cc.dcm',
'ID_422f2baa2.dcm',
'ID_2e6be2bf4.dcm',
'ID_401502625.dcm',
'ID_0bd436685.dcm',
'ID_46a16f1fe.dcm',
'ID_eb6ab0790.dcm',
'ID_173f5d1ab.dcm',
'ID_213560b6d.dcm',
'ID_c7d36ab7b.dcm',
'ID_b8c40e801.dcm',
'ID_236dd7127.dcm',
'ID_0b0d0fb0b.dcm',
'ID_5d4c63ddf.dcm',
'ID_3679f8287.dcm',
'ID_725c19675.dcm',
'ID_b54407339.dcm',
'ID_26a71e6d1.dcm',
'ID_24867ad69.dcm',
'ID_0d3bb8c23.dcm',
'ID_7adf34a33.dcm',
'ID_06c079085.dcm',
'ID_6960a631e.dcm',
'ID_bb12d1061.dcm',
'ID_7541d0d95.dcm',
'ID_71a01eb16.dcm',
'ID_ae06afcca.dcm',
'ID_308fd438c.dcm',
'ID_5999aa580.dcm',
'ID_011b2c8ab.dcm',
'ID_2ec0f1c8e.dcm',
'ID_f15f4e570.dcm',
'ID_9ce4a1e35.dcm',
'ID_35435e48c.dcm',
'ID_5f0c5a633.dcm',
'ID_7b1947763.dcm',
'ID_7150d9785.dcm',
'ID_eca14887c.dcm',
'ID_9ad5e2127.dcm',
'ID_874e1cd2a.dcm',
'ID_20f03c5f8.dcm',
'ID_3df0b0bbd.dcm',
'ID_19b04e16c.dcm',
'ID_ed367b4af.dcm',
'ID_69b327a8d.dcm',
'ID_98c27d5b4.dcm',
'ID_49bec9cc1.dcm',
'ID_66435449f.dcm',
'ID_0971561e3.dcm',
'ID_f6a6c9cc3.dcm',
'ID_4eb955b3a.dcm',
'ID_d6802d02f.dcm',
'ID_46e0fc031.dcm',
'ID_a33f01f58.dcm',
'ID_7ce0cf066.dcm',
'ID_7f61410df.dcm',
'ID_edadea596.dcm',
'ID_4892fc8c5.dcm',
'ID_48a5db7b0.dcm',
'ID_f13314cdc.dcm',
'ID_9cc7e8bad.dcm',
'ID_1452ccb1e.dcm',
'ID_5b953f708.dcm',
'ID_32b0c8277.dcm',
'ID_ba9fa6de9.dcm',
'ID_2d0a1bbbe.dcm',
'ID_e6088e970.dcm',
'ID_0b489db24.dcm',
'ID_42b468f1f.dcm',
'ID_30bdbd782.dcm',
'ID_90f594b0d.dcm',
'ID_bdeb87de7.dcm',
'ID_5d424f84e.dcm',
'ID_c96ba4788.dcm',
'ID_fdd80628c.dcm',
'ID_2b1bfe4f0.dcm',
'ID_10d96e8b6.dcm',
'ID_c3d97977e.dcm',
'ID_a6a683aa5.dcm',
'ID_ba1d2a45f.dcm',
'ID_dc528564e.dcm',
'ID_f81053c01.dcm',
'ID_dfe313025.dcm',
'ID_f61e27fe0.dcm',
'ID_f520296a9.dcm',
'ID_cbfcebc66.dcm',
'ID_a2c161640.dcm',
'ID_17deb2f66.dcm',
'ID_25963b03b.dcm',
'ID_94baeb8fa.dcm',
'ID_233e28af7.dcm',
'ID_cc3863c09.dcm',
'ID_1376f263b.dcm',
'ID_ef426964f.dcm',
'ID_9ec8d71b3.dcm',
'ID_e83d46363.dcm',
'ID_1644d2e4c.dcm',
'ID_6b21c4fb1.dcm',
'ID_48ac70322.dcm',
'ID_fff5b1961.dcm',
'ID_0d6ee318b.dcm',
'ID_1b9c3ec0c.dcm',
'ID_8f75bbc9c.dcm',
'ID_f3af8e0b5.dcm',
'ID_d3b67b70b.dcm',
'ID_88eaa8286.dcm',
'ID_ecd792242.dcm',
'ID_41c28506a.dcm',
'ID_3b69852d0.dcm',
'ID_943db700f.dcm',
'ID_06d8afb17.dcm',
'ID_5bbcfc0a7.dcm',
'ID_484a75956.dcm',
'ID_4ff746023.dcm',
'ID_21618f60f.dcm',
'ID_05e912d2a.dcm',
'ID_93579c286.dcm',
'ID_f6cb89a5b.dcm',
'ID_c3d1cca30.dcm',
'ID_f8284fbd9.dcm',
'ID_1be4a27dd.dcm',
'ID_810a42b9d.dcm',
'ID_c930cb2ca.dcm',
'ID_8a1d535e8.dcm',
'ID_b1dddf2ca.dcm',
'ID_8a79effdb.dcm',
'ID_0b4e7966c.dcm',
'ID_09ad7f651.dcm',
'ID_c002497a5.dcm',
'ID_7321129e5.dcm',
'ID_9254e5c15.dcm',
'ID_bf327dd70.dcm',
'ID_5e84630c3.dcm',
'ID_c2cd9395b.dcm',
'ID_f0c3677d0.dcm',
'ID_73df1c31a.dcm',
'ID_2ce060c1d.dcm',
'ID_f35fc6059.dcm',
'ID_3c68cfad2.dcm',
'ID_c9c23e7b4.dcm',
'ID_17ad86517.dcm',
'ID_46690fea2.dcm',
'ID_6be68c696.dcm',
'ID_8c4684328.dcm',
'ID_64c4cdaa8.dcm',
'ID_a83e397d6.dcm',
'ID_7279968e1.dcm',
'ID_9eaed6a86.dcm',
'ID_6b70c6a35.dcm',
'ID_fb0be4a98.dcm',
'ID_947193dca.dcm',
'ID_003f3aae6.dcm',
'ID_d5360dabf.dcm',
'ID_8cbc44c9f.dcm',
'ID_b4637d195.dcm',
'ID_bac813fb8.dcm',
'ID_9f869fa25.dcm',
'ID_9a6a17078.dcm',
'ID_04fdb569c.dcm',
'ID_287c31784.dcm',
'ID_62147bd89.dcm',
'ID_5b13b009a.dcm',
'ID_a0cfa8e3b.dcm',
'ID_1f4872e82.dcm',
'ID_2f87a17e8.dcm',
'ID_695e265f4.dcm',
'ID_03b56a971.dcm',
'ID_c552d94e2.dcm',
'ID_7fb7489ff.dcm',
'ID_ed4064eea.dcm',
'ID_c261d78d4.dcm',
'ID_3fa608f4d.dcm',
'ID_1761bb532.dcm',
'ID_4e322cf42.dcm',
'ID_e0bf17997.dcm',
'ID_0ce41f17f.dcm',
'ID_d55e8dd7c.dcm',
'ID_ed2906f4b.dcm',
'ID_776d942e1.dcm',
'ID_9698dbd62.dcm',
'ID_c689ea819.dcm',
'ID_7e756d3f8.dcm',
'ID_7e4f87513.dcm',
'ID_1f6ab564f.dcm',
'ID_8e6ae69c9.dcm',
'ID_ed6bb230b.dcm',
'ID_1ec8e1159.dcm',
'ID_c35c310ca.dcm',
'ID_37ace04dc.dcm',
'ID_1b9d11ccb.dcm',
'ID_25e85f17b.dcm',
'ID_18c868633.dcm',
'ID_0fe90a90f.dcm',
'ID_0cff5cc5e.dcm',
'ID_e460e3d18.dcm',
'ID_5c393a543.dcm',
'ID_931285022.dcm',
'ID_cd9cee36e.dcm',
'ID_a2eb50f3e.dcm',
'ID_a53d1d0f6.dcm',
'ID_dee59c957.dcm',
'ID_0570e1112.dcm',
'ID_e62c79995.dcm',
'ID_d96375ad0.dcm',
'ID_8a9be71d3.dcm',
'ID_84482f3d7.dcm',
'ID_b13d5cff5.dcm',
'ID_1c157b42f.dcm',
'ID_9e699cf47.dcm',
'ID_dceb7fafa.dcm',
'ID_8c1bf2f13.dcm',
'ID_1cfb57648.dcm',
'ID_7ffb4c8eb.dcm',
'ID_289a8ccdd.dcm',
'ID_7c02f194f.dcm',
'ID_fabdb6988.dcm',
'ID_0176360b8.dcm',
'ID_2c63b796e.dcm',
'ID_79eb10468.dcm',
'ID_97046a9aa.dcm',
'ID_aae9d8c6a.dcm',
'ID_1d6439c07.dcm',
'ID_62c9f56af.dcm',
'ID_f4db57051.dcm',
'ID_969b45988.dcm',
'ID_58ed11592.dcm',
'ID_826627d9f.dcm',
'ID_b4dad2afc.dcm',
'ID_21914da05.dcm',
'ID_37e38c5b8.dcm',
'ID_e2c88c1a9.dcm',
'ID_439dca032.dcm',
'ID_88772d775.dcm',
'ID_369a9d660.dcm',
'ID_28c38d39b.dcm',
'ID_1a83d2b7b.dcm',
'ID_541f7363c.dcm',
'ID_bbab4b545.dcm',
'ID_aacb4896e.dcm',
'ID_d3e793d97.dcm',
'ID_2d6d230f3.dcm',
'ID_930a0ee3a.dcm',
'ID_a56e90b5e.dcm',
'ID_d31756cfd.dcm',
'ID_ee3975eb8.dcm',
'ID_4d8415720.dcm',
'ID_6a92ed9b0.dcm',
'ID_35d5735ba.dcm',
'ID_abc531ef0.dcm',
'ID_14ae328e1.dcm',
'ID_52dcc3102.dcm',
'ID_9eb748dff.dcm',
'ID_516081726.dcm',
'ID_e68fc7076.dcm',
'ID_ed2487f30.dcm',
'ID_26a3e0206.dcm',
'ID_39d8ea904.dcm',
'ID_de3ad1845.dcm',
'ID_00732f1c0.dcm',
'ID_4d885686e.dcm',
'ID_792b8cf07.dcm',
'ID_cda9c493c.dcm',
'ID_04a594fbe.dcm',
'ID_c9ec17cf6.dcm',
'ID_7fc3c1d18.dcm',
'ID_4b0f46337.dcm',
'ID_a8196113b.dcm',
'ID_a6547cf0c.dcm',
'ID_598cc1c08.dcm',
'ID_5cb894560.dcm',
'ID_26b737f28.dcm',
'ID_720861abd.dcm',
'ID_e90963c0a.dcm',
'ID_038ec4941.dcm',
'ID_9a75eca76.dcm',
'ID_f2f163bf7.dcm',
'ID_d69890fa2.dcm',
'ID_d1520adb9.dcm',
'ID_0c0622d78.dcm',
'ID_390a7b383.dcm',
'ID_06a9d8c0c.dcm',
'ID_d00607347.dcm',
'ID_eabda2d51.dcm',
'ID_cd3b53d82.dcm',
'ID_58cc53da9.dcm',
'ID_cd300bc35.dcm',
'ID_8f131f37d.dcm',
'ID_3e05d4eb5.dcm',
'ID_299005658.dcm',
'ID_8c14a0a1f.dcm',
'ID_5c76676ff.dcm',
'ID_38319ea56.dcm',
'ID_54763e1ba.dcm',
'ID_f729f7387.dcm',
'ID_581926253.dcm',
'ID_c740ccf10.dcm',
'ID_4060c4f47.dcm',
'ID_44d1b3f64.dcm',
'ID_9c6b5ab88.dcm',
'ID_6c9eedfa6.dcm',
'ID_10928400d.dcm',
'ID_f2e819867.dcm',
'ID_c83ae89e6.dcm',
'ID_981c6a915.dcm',
'ID_61c8bc119.dcm',
'ID_20ce0d600.dcm',
'ID_4656436bb.dcm',
'ID_1846b4a3e.dcm',
'ID_27b0adf97.dcm',
'ID_21a760b1a.dcm',
'ID_42a0c2a86.dcm',
'ID_d9364564b.dcm',
'ID_732f5c923.dcm',
'ID_a987cf7e5.dcm',
'ID_a56a6ca53.dcm',
'ID_3e9afd76d.dcm',
'ID_066b44ea5.dcm',
'ID_c69598bef.dcm',
'ID_19a812260.dcm',
'ID_ea34e9a29.dcm',
'ID_80f80f237.dcm',
'ID_092295356.dcm',
'ID_f0edbacbd.dcm',
'ID_ee6f09fbd.dcm',
'ID_fbee5d185.dcm',
'ID_69d6e65ff.dcm',
'ID_2a3f79825.dcm',
'ID_15bb21539.dcm',
'ID_e38017781.dcm',
'ID_84819f7ec.dcm',
'ID_af3a71504.dcm',
'ID_cfd027b8f.dcm',
'ID_11b5e4d9a.dcm',
'ID_dbf68bd58.dcm',
'ID_e0b7018b1.dcm',
'ID_16e1d5e78.dcm',
'ID_03019a9bc.dcm',
'ID_6adfbaeb5.dcm',
'ID_aa4cc1c72.dcm',
'ID_0ecd3a2fb.dcm',
'ID_977f1708e.dcm',
'ID_371d232bb.dcm',
'ID_1c05040eb.dcm',
'ID_d2d0602b0.dcm',
'ID_613b5c760.dcm',
'ID_cf6b5300f.dcm',
'ID_49194983b.dcm',
'ID_96f181da2.dcm',
'ID_a0fa2a232.dcm',
'ID_d084c9655.dcm',
'ID_6c1eaf40b.dcm',
'ID_eef65ec58.dcm',
'ID_c0a927636.dcm',
'ID_bf684705b.dcm',
'ID_9c1f7fab3.dcm',
'ID_75afccb11.dcm',
'ID_d8ca8b64e.dcm',
'ID_4d52efec5.dcm',
'ID_d59616d1f.dcm',
'ID_4fe4d648f.dcm',
'ID_07f6d5802.dcm',
'ID_b5fe0b31a.dcm',
'ID_5a1815676.dcm',
'ID_7ecca0411.dcm',
'ID_e0cdfc1ad.dcm',
'ID_4f920075a.dcm',
'ID_d4f281705.dcm',
'ID_cd1c021c0.dcm',
'ID_ef39740c8.dcm',
'ID_a28f3c683.dcm',
'ID_97d6d88d6.dcm',
'ID_91af67234.dcm',
'ID_ff9896921.dcm',
'ID_b7a0bb435.dcm',
'ID_3a201f5ba.dcm',
'ID_0b9d8082c.dcm',
'ID_f8f85973b.dcm',
'ID_609e38a10.dcm',
'ID_244ca0cfa.dcm',
'ID_18a0235a3.dcm',
'ID_66072acb4.dcm',
'ID_166812611.dcm',
'ID_378a41258.dcm',
'ID_25c02f22f.dcm',
'ID_82debd520.dcm',
'ID_0c46dff60.dcm',
'ID_0117aa276.dcm',
'ID_a5abe9a5e.dcm',
'ID_4136f0580.dcm',
'ID_02706475a.dcm',
'ID_a0e6141a2.dcm',
'ID_cceb3f6e3.dcm',
'ID_1bbac911a.dcm',
'ID_dc32b48f6.dcm',
'ID_25c9c5b99.dcm',
'ID_a2b1c3cd3.dcm',
'ID_358cc70b6.dcm',
'ID_38213fe3e.dcm',
'ID_23103d583.dcm',
'ID_af1a10829.dcm',
'ID_a27065852.dcm',
'ID_9b3bad092.dcm',
'ID_e7016c9ac.dcm',
'ID_267502639.dcm',
'ID_347a7f129.dcm',
'ID_8b68af4e9.dcm',
'ID_4d88ff3aa.dcm',
'ID_d97880a3b.dcm',
'ID_5558ec7e9.dcm',
'ID_ac187c289.dcm',
'ID_9604f6ca0.dcm',
'ID_3608a5e3f.dcm',
'ID_c4c8b264b.dcm',
'ID_21c9f2b12.dcm',
'ID_587a90e44.dcm',
'ID_66218f652.dcm',
'ID_63b2a08ab.dcm',
'ID_030a8b768.dcm',
'ID_48b3746ee.dcm',
'ID_37741d984.dcm',
'ID_bef8cb4e1.dcm',
'ID_2e6825e63.dcm',
'ID_c019b1cc2.dcm',
'ID_792e458b6.dcm',
'ID_4b2b31863.dcm',
'ID_35c034da3.dcm',
'ID_555d36a18.dcm',
'ID_b68554737.dcm',
'ID_738e6c973.dcm',
'ID_c7804b2fc.dcm',
'ID_4a58e3baf.dcm',
'ID_86ea194b7.dcm',
'ID_cf0b774e0.dcm',
'ID_8df25523d.dcm',
'ID_9931de0be.dcm',
'ID_40b12e717.dcm',
'ID_3cad5af18.dcm',
'ID_9943c830b.dcm',
'ID_847824ff9.dcm',
'ID_0aee09969.dcm',
'ID_4add69ce4.dcm',
'ID_9de6d2987.dcm',
'ID_7da6f9618.dcm',
'ID_b1e2024d5.dcm',
'ID_21052c946.dcm',
'ID_f2e026211.dcm',
'ID_387cfd441.dcm',
'ID_834fbd09b.dcm',
'ID_8bd951807.dcm',
'ID_7b630b97f.dcm',
'ID_31c72a798.dcm',
'ID_f795c243f.dcm',
'ID_013a90c6c.dcm',
'ID_b7aca5929.dcm',
'ID_8bcbf0003.dcm',
'ID_30d718587.dcm',
'ID_ee409bee6.dcm',
'ID_8e1b5ba41.dcm',
'ID_89a6b5d13.dcm',
'ID_84a00ef58.dcm',
'ID_5a1791de2.dcm',
'ID_923769223.dcm',
'ID_3dc6a0088.dcm',
'ID_5308f3b30.dcm',
'ID_e6e31f575.dcm',
'ID_e95353b19.dcm',
'ID_ff60898b7.dcm',
'ID_806035cd5.dcm',
'ID_a319e806c.dcm',
'ID_6aafdda34.dcm',
'ID_5fe292f25.dcm',
'ID_b473cc505.dcm',
'ID_3bdc8d020.dcm',
'ID_c2176e9e5.dcm',
'ID_2a6a7dfce.dcm',
'ID_401a927f3.dcm',
'ID_35fe179c5.dcm',
'ID_b9ac992ef.dcm',
'ID_230024e9f.dcm',
'ID_183d81fb5.dcm',
'ID_2391fabf9.dcm',
'ID_e5ec5688f.dcm',
'ID_72024cb05.dcm',
'ID_d505f24fc.dcm',
'ID_611806300.dcm',
'ID_06ac65479.dcm',
'ID_d618db701.dcm',
'ID_13cb8d30c.dcm',
'ID_90f502f91.dcm',
'ID_54e597f3c.dcm',
'ID_d6e926954.dcm',
'ID_3d747c3bf.dcm',
'ID_c47517690.dcm',
'ID_ef79ae79d.dcm',
'ID_f66d68959.dcm',
'ID_a365347d1.dcm',
'ID_d41f59c78.dcm',
'ID_f61d2c0fa.dcm',
'ID_282e70cce.dcm',
'ID_5059d4771.dcm',
'ID_9f671171e.dcm',
'ID_05b9ae449.dcm',
'ID_423c351e6.dcm',
'ID_6feec890a.dcm',
'ID_bc199e25a.dcm',
'ID_9e7b7c092.dcm',
'ID_8791e9c8e.dcm',
'ID_69e9cc863.dcm',
'ID_5c5e0ef1a.dcm',
'ID_832360236.dcm',
'ID_49670e913.dcm',
'ID_3dac1a969.dcm',
'ID_f91661548.dcm',
'ID_42243af88.dcm',
'ID_022293788.dcm',
'ID_e628c38f0.dcm',
'ID_183e248e6.dcm',
'ID_963768077.dcm',
'ID_a2c7fa36a.dcm',
'ID_505917592.dcm',
'ID_7068635c6.dcm',
'ID_b539166bd.dcm',
'ID_4421219b3.dcm',
'ID_423881dc6.dcm',
'ID_2bafced74.dcm',
'ID_6f8357a5f.dcm',
'ID_7a6d9fdef.dcm',
'ID_fc1537a6f.dcm',
'ID_62a52071c.dcm',
'ID_c5edb0cc5.dcm',
'ID_8183e3a48.dcm',
'ID_9dd1be051.dcm',
'ID_d4fd3b97b.dcm',
'ID_f5295dcae.dcm',
'ID_d488e8f8c.dcm',
'ID_ca9bb984a.dcm',
'ID_7a26e9b62.dcm',
'ID_da54a3b73.dcm',
'ID_ea85e0eab.dcm',
'ID_ddf11a817.dcm',
'ID_6d04ed26f.dcm',
'ID_64a82004f.dcm',
'ID_e7b4ab488.dcm',
'ID_305aee9a2.dcm',
'ID_787cf8766.dcm',
'ID_d2e606a96.dcm',
'ID_12e6a5e75.dcm',
'ID_ca106588c.dcm',
'ID_bd2224320.dcm',
'ID_14d54c3c8.dcm',
'ID_34fbfb0d6.dcm',
'ID_8a4866a5e.dcm',
'ID_e40eebb3e.dcm',
'ID_995e1f87c.dcm',
'ID_a8f12c28e.dcm',
'ID_dbd9d86da.dcm',
'ID_57f37af96.dcm',
'ID_e44158b97.dcm',
'ID_b1f947c73.dcm',
'ID_47ee35242.dcm',
'ID_7657f9e71.dcm',
'ID_173242a64.dcm',
'ID_f0a84faee.dcm',
'ID_1eba28643.dcm',
'ID_7a0684245.dcm',
'ID_ec0063a25.dcm',
'ID_fa09414d5.dcm',
'ID_9d3362f09.dcm',
'ID_649298eae.dcm',
'ID_185df5a88.dcm',
'ID_12cadc6af.dcm',
'ID_38fd7baa0.dcm',
'ID_6c5d82413.dcm',
'ID_aec8e68b3.dcm',
'ID_4d9209c7c.dcm'

]

In [12]:
gcs_data_path = "gs://rsna_hemorrhage/dataset/intraparenchymal_negative_3 channels2"

In [13]:
gcsfuse_mount_data_path_PNG = resolve_gcs_uri_to_local_uri(gcs_file_uri=gcs_data_path)

In [14]:
for name in names:
    
    # Get the image data
    # Open the DICOM file
    #ds = pydicom.read_file("dataset_stage_2_train_ID_00008ce3c.dcm")
    im=dcm.dcmread(gcsfuse_mount_data_path+'/'+name)
    image = im.pixel_array
    hu_image = transform_to_hu(im, image)
    brain_image = window_image(hu_image, 40, 80)
    subdural_image = window_image(hu_image, 80, 200)
    soft_image = window_image(hu_image, 40, 380)

    brain_image = (brain_image - 0) / 80
    subdural_image = (subdural_image - (-20)) / 200
    soft_image = (soft_image - (-150)) / 380
    
    combined_image = np.array([brain_image, subdural_image, soft_image]).transpose(1, 2, 0)
    
    #combined_img = pilim.fromarray(combined_img)
    #image.save(gcsfuse_mount_data_path_PNG+'/'+name[:-4]+'.png')
    plt.imsave(gcsfuse_mount_data_path_PNG+'/'+name[:-4]+'.png',  combined_image, cmap=plt.cm.gray)
    #image = ds.pixel_array

In [15]:
#import pandas as pd
#df=pd.DataFrame(names)

In [16]:
#df.to_csv('names_train_png.csv') 

In [17]:
PNGFiles_train = [] 
for dirName, subdirList, fileList in os.walk(gcsfuse_mount_data_path_PNG):
    for filename in fileList:
        if ".png" in filename.lower():
            PNGFiles_train.append(os.path.join(dirName,filename))
print("Number of train (.png) files =", len(PNGFiles_train))

Number of train (.png) files = 1005


In [18]:
#gcs_data_path = "gs://rsna_hemorrhage/dataset"

In [19]:
#gcsfuse_mount_data_path = resolve_gcs_uri_to_local_uri(gcs_file_uri=gcs_data_path)

In [20]:
#os.mkdir(f"{gcsfuse_mount_data_path}/test_PNG")

In [21]:
#os.mkdir(f"{gcsfuse_mount_data_path}/train_PNG")

In [22]:
#image.save(f"{gcsfuse_mount_data_path}/train_PNG/name+'.png'")

In [23]:
#os.listdir(gcsfuse_mount_data_path.split("/Images")[0])

In [24]:
#def _bytes_feature(value):
        #"""Returns a bytes_list from a string / byte."""
        #if isinstance(value, type(tf.constant(0))):
           # value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
       # return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
#def _float_feature(value):
    #"""Returns a float_list from a float / double."""
    #return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

#def _int64_feature(value):
    #"""Returns an int64_list from a bool / enum / int / uint."""
    #return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [25]:
#def create_tfrecords(img_path, tfrecords_path):
    #img = Image.open(img_path)
    #full_tfrecord_path = f"{tfrecords_path}/"

In [26]:
#gcsfuse_mount_data_path = resolve_gcs_uri_to_local_uri(gcs_file_uri=gcs_data_path)

In [27]:
#os.mkdir(f"{gcsfuse_mount_data_path}/test_PNG")

In [28]:
#os.mkdir(f"{gcsfuse_mount_data_path}/train_PNG")

In [29]:
#image.save(f"{gcsfuse_mount_data_path}/train_PNG/name+'.png'")

In [30]:
#os.listdir(gcsfuse_mount_data_path.split("/Images")[0])

In [31]:
#def _bytes_feature(value):
        #"""Returns a bytes_list from a string / byte."""
        #if isinstance(value, type(tf.constant(0))):
           # value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
       # return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
#def _float_feature(value):
    #"""Returns a float_list from a float / double."""
    #return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

#def _int64_feature(value):
    #"""Returns an int64_list from a bool / enum / int / uint."""
    #return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [32]:
#def create_tfrecords(img_path, tfrecords_path):
    #img = Image.open(img_path)
    #full_tfrecord_path = f"{tfrecords_path}/"